In [ ]:
#Description: This program predicts if the stock price of a company will increase or decrease
#based on top news headlines.

In [ ]:
pip install vaderSentiment

     |████████████████████████████████| 133kB 5.0MB/s 


In [ ]:
import pandas as pd
import numpy as np
from textblob import TextBlob
import re
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.ensemble import RandomForestClassifier


In [ ]:
#store data
df1=pd.read_csv('/content/News_DJIA.csv');
df2=pd.read_csv('/content/Value_DJIA.csv');

In [ ]:
#print the first 3 rows of data for df1
df1.head(3)

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,Top9,Top10,Top11,Top12,Top13,Top14,Top15,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...","b""The 'enemy combatent' trials are nothing but...",b'Georgian troops retreat from S. Osettain cap...,b'Did the U.S. Prep Georgia for War with Russia?',b'Rice Gives Green Light for Israel to Attack ...,b'Announcing:Class Action Lawsuit on Behalf of...,"b""So---Russia and Georgia are at war and the N...","b""China tells Bush to stay out of other countr...",b'Did World War III start today?',b'Georgia Invades South Ossetia - if Russia ge...,b'Al-Qaeda Faces Islamist Backlash',"b'Condoleezza Rice: ""The US would not act to p...",b'This is a busy day: The European Union has ...,"b""Georgia will withdraw 1,000 soldiers from Ir...",b'Why the Pentagon Thinks Attacking Iran is a ...,b'Caucasus in crisis: Georgia invades South Os...,b'Indian shoe manufactory - And again in a se...,b'Visitors Suffering from Mental Illnesses Ban...,"b""No Help for Mexico's Kidnapping Surge"""
1,2008-08-11,1,b'Why wont America and Nato help us? If they w...,b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli ...",b'Georgian army flees in disarray as Russians ...,"b""Olympic opening ceremony fireworks 'faked'""",b'What were the Mossad with fraudulent New Zea...,b'Russia angered by Israeli military sale to G...,b'An American citizen living in S.Ossetia blam...,b'Welcome To World War IV! Now In High Definit...,"b""Georgia's move, a mistake of monumental prop...",b'Russia presses deeper into Georgia; U.S. say...,b'Abhinav Bindra wins first ever Individual Ol...,b' U.S. ship heads for Arctic to define territ...,b'Drivers in a Jerusalem taxi station threaten...,b'The French Team is Stunned by Phelps and the...,b'Israel and the US behind the Georgian aggres...,"b'""Do not believe TV, neither Russian nor Geor...",b'Riots are still going on in Montreal (Canada...,b'China to overtake US as largest manufacturer',b'War in South Ossetia [PICS]',b'Israeli Physicians Group Condemns State Tort...,b' Russia has just beaten the United States ov...,b'Perhaps *the* question about the Georgia - R...,b'Russia is so much better at war',"b""So this is what it's come to: trading sex fo..."
2,2008-08-12,0,b'Remember that adorable 9-year-old who sang a...,"b""Russia 'ends Georgia operation'""","b'""If we had no sexual harassment we would hav...","b""Al-Qa'eda is losing support in Iraq because ...",b'Ceasefire in Georgia: Putin Outmaneuvers the...,b'Why Microsoft and Intel tried to kill the XO...,b'Stratfor: The Russo-Georgian War and the Bal...,"b""I'm Trying to Get a Sense of This Whole Geor...","b""The US military was surprised by the timing ...",b'U.S. Beats War Drum as Iran Dumps the Dollar',"b'Gorbachev: ""Georgian military attacked the S...",b'CNN use footage of Tskhinvali ruins to cover...,b'Beginning a war as the Olympics were opening...,b'55 pyramids as large as the Luxor stacked in...,b'The 11 Top Party Cities in the World',b'U.S. troops still in Georgia (did you know t...,b'Why Russias response to Georgia was right',"b'Gorbachev accuses U.S. of making a ""serious ...","b'Russia, Georgia, and NATO: Cold War Two'",b'Remember that adorable 62-year-old who led y...,b'War in Georgia: The Israeli connection',b'All signs point to the US encouraging Georgi...,b'Christopher King argues that the US and NATO...,b'America: The New Mexico?',"b""BBC NEWS | Asia-Pacific | Extinction 'by man..."


In [ ]:
#Get number of rows and cols
df1.shape

(1989, 27)

In [ ]:
#print the first 3 rows of data for df2
df2.head(3)

,Date,Open,High,Low,Close,Adj Close,Volume
0,2008-01-02,13261.820313,13279.540039,12991.370117,13043.959961,13043.959961,239580000
1,2008-01-03,13044.120117,13137.929688,13023.559570,13056.719727,13056.719727,200620000
2,2008-01-04,13046.559570,13046.719727,12789.040039,12800.179688,12800.179688,304210000


In [ ]:
df2.shape


(2266, 7)

In [ ]:
#merge the data set on the date field
merge=df1.merge(df2,how='inner',on='Date',left_index=False)

In [ ]:
merge.index

Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
            ...
            1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988],
           dtype='int64', length=1989)

In [ ]:
#combining the top news headline
headlines=[]

for row in range(0,len(merge.index)):
  headlines.append(' '.join(str(x)for x in merge.iloc[row,2:27]))

In [ ]:
headlines[0]

'b"Georgia \'downs two Russian warplanes\' as countries move to brink of war" b\'BREAKING: Musharraf to be impeached.\' b\'Russia Today: Columns of troops roll into South Ossetia; footage from fighting (YouTube)\' b\'Russian tanks are moving towards the capital of South Ossetia, which has reportedly been completely destroyed by Georgian artillery fire\' b"Afghan children raped with \'impunity,\' U.N. official says - this is sick, a three year old was raped and they do nothing" b\'150 Russian tanks have entered South Ossetia whilst Georgia shoots down two Russian jets.\' b"Breaking: Georgia invades South Ossetia, Russia warned it would intervene on SO\'s side" b"The \'enemy combatent\' trials are nothing but a sham: Salim Haman has been sentenced to 5 1/2 years, but will be kept longer anyway just because they feel like it." b\'Georgian troops retreat from S. Osettain capital, presumably leaving several hundred people killed. [VIDEO]\' b\'Did the U.S. Prep Georgia for War with Russia?\'

In [ ]:
#clean the data
clean_headlines=[]
for i in range(0,len(headlines)):
  clean_headlines.append(re.sub("b[(')]",'',headlines[i]))
  clean_headlines[i]=(re.sub('b[(")]','',clean_headlines[i]))
  clean_headlines[i]=(re.sub("\'",'',clean_headlines[i]))

In [ ]:
clean_headlines[0]

'Georgia downs two Russian warplanes as countries move to brink of war" BREAKING: Musharraf to be impeached. Russia Today: Columns of troops roll into South Ossetia; footage from fighting (YouTube) Russian tanks are moving towards the capital of South Ossetia, which has reportedly been completely destroyed by Georgian artillery fire Afghan children raped with impunity, U.N. official says - this is sick, a three year old was raped and they do nothing" 150 Russian tanks have entered South Ossetia whilst Georgia shoots down two Russian jets. Breaking: Georgia invades South Ossetia, Russia warned it would intervene on SOs side" The enemy combatent trials are nothing but a sham: Salim Haman has been sentenced to 5 1/2 years, but will be kept longer anyway just because they feel like it." Georgian troops retreat from S. Osettain capital, presumably leaving several hundred people killed. [VIDEO] Did the U.S. Prep Georgia for War with Russia? Rice Gives Green Light for Israel to Attack Iran: S

In [ ]:
#add clean headlines to merge data
merge['Combined_News']=clean_headlines

In [ ]:
merge['Combined_News'][0]

'Georgia downs two Russian warplanes as countries move to brink of war" BREAKING: Musharraf to be impeached. Russia Today: Columns of troops roll into South Ossetia; footage from fighting (YouTube) Russian tanks are moving towards the capital of South Ossetia, which has reportedly been completely destroyed by Georgian artillery fire Afghan children raped with impunity, U.N. official says - this is sick, a three year old was raped and they do nothing" 150 Russian tanks have entered South Ossetia whilst Georgia shoots down two Russian jets. Breaking: Georgia invades South Ossetia, Russia warned it would intervene on SOs side" The enemy combatent trials are nothing but a sham: Salim Haman has been sentenced to 5 1/2 years, but will be kept longer anyway just because they feel like it." Georgian troops retreat from S. Osettain capital, presumably leaving several hundred people killed. [VIDEO] Did the U.S. Prep Georgia for War with Russia? Rice Gives Green Light for Israel to Attack Iran: S

In [ ]:
def getSubjectivity(text):
  return TextBlob(text).sentiment.subjectivity

def getPolarity(text):
  return TextBlob(text).sentiment.polarity

In [ ]:
merge['Subjectivity']=merge['Combined_News'].apply(getSubjectivity)
merge["Polarity"]=merge['Combined_News'].apply(getPolarity)

In [ ]:
  def getSIA(text):
    sia=SentimentIntensityAnalyzer()
    sentiment=sia.polarity_scores(text)
    return sentiment

In [ ]:
compound=[]
neg=[]
pos=[]
neu=[]
SIA=0

for i in range(0,len(merge["Combined_News"])):
  SIA=getSIA(merge["Combined_News"][i])
  compound.append(SIA['compound'])
  neg.append(SIA['neg'])
  pos.append(SIA['pos'])
  neu.append(SIA['neu'])

In [ ]:
merge["compound"]=compound
merge["positive"]=pos
merge["negative"]=neg
merge["neutral"]=neu

In [ ]:
merge.head(3)

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,Top9,Top10,Top11,Top12,Top13,Top14,Top15,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25,Open,High,Low,Close,Adj Close,Volume,Combined_News,Subjectivity,Polarity,compound,positive,negative,neutral
0,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...","b""The 'enemy combatent' trials are nothing but...",b'Georgian troops retreat from S. Osettain cap...,b'Did the U.S. Prep Georgia for War with Russia?',b'Rice Gives Green Light for Israel to Attack ...,b'Announcing:Class Action Lawsuit on Behalf of...,"b""So---Russia and Georgia are at war and the N...","b""China tells Bush to stay out of other countr...",b'Did World War III start today?',b'Georgia Invades South Ossetia - if Russia ge...,b'Al-Qaeda Faces Islamist Backlash',"b'Condoleezza Rice: ""The US would not act to p...",b'This is a busy day: The European Union has ...,"b""Georgia will withdraw 1,000 soldiers from Ir...",b'Why the Pentagon Thinks Attacking Iran is a ...,b'Caucasus in crisis: Georgia invades South Os...,b'Indian shoe manufactory - And again in a se...,b'Visitors Suffering from Mental Illnesses Ban...,"b""No Help for Mexico's Kidnapping Surge""",11432.089844,11759.959961,11388.040039,11734.320313,11734.320313,212830000,Georgia downs two Russian warplanes as countri...,0.267549,-0.048568,-0.9982,0.041,0.235,0.724
1,2008-08-11,1,b'Why wont America and Nato help us? If they w...,b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli ...",b'Georgian army flees in disarray as Russians ...,"b""Olympic opening ceremony fireworks 'faked'""",b'What were the Mossad with fraudulent New Zea...,b'Russia angered by Israeli military sale to G...,b'An American citizen living in S.Ossetia blam...,b'Welcome To World War IV! Now In High Definit...,"b""Georgia's move, a mistake of monumental prop...",b'Russia presses deeper into Georgia; U.S. say...,b'Abhinav Bindra wins first ever Individual Ol...,b' U.S. ship heads for Arctic to define territ...,b'Drivers in a Jerusalem taxi station threaten...,b'The French Team is Stunned by Phelps and the...,b'Israel and the US behind the Georgian aggres...,"b'""Do not believe TV, neither Russian nor Geor...",b'Riots are still going on in Montreal (Canada...,b'China to overtake US as largest manufacturer',b'War in South Ossetia [PICS]',b'Israeli Physicians Group Condemns State Tort...,b' Russia has just beaten the United States ov...,b'Perhaps *the* question about the Georgia - R...,b'Russia is so much better at war',"b""So this is what it's come to: trading sex fo...",11729.669922,11867.110352,11675.530273,11782.349609,11782.349609,183190000,Why wont America and Nato help us? If they won...,0.374806,0.121956,-0.9858,0.089,0.191,0.721
2,2008-08-12,0,b'Remember that adorable 9-year-old who sang a...,"b""Russia 'ends Georgia operation'""","b'""If we had no sexual harassment we would hav...","b""Al-Qa'eda is losing support in Iraq because ...",b'Ceasefire in Georgia: Putin Outmaneuvers the...,b'Why Microsoft and Intel tried to kill the XO...,b'Stratfor: The Russo-Georgian War and the Bal...,"b""I'm Trying to Get a Sense of This Whole Geor...","b""The US military was surprised by the timing ...",b'U.S. Beats War Drum as Iran Dumps the Dollar',"b'Gorbachev: ""Georgian military attacked the S...",b'CNN use footage of Tskhinvali ruins to cover...,b'Beginning a war as the Olympics were opening...,b'55 pyramids as large as the Luxor stacked in...,b'The 11 Top Party Cities in the World',b'U.S. troops still in Georgia (did you know t...,b'Why Russias response to Georgia was right',"b'Gorbachev accuses U.S. of making a ""serious ...","b'Russia, Georgia

In [ ]:
keep_columns=['Open',	'High',	'Low',	'Volume',		'Subjectivity',	'Polarity',	'compound',	'positive',	'negative',	'neutral','Label']
df=merge[keep_columns]
df

,Open,High,Low,Volume,Subjectivity,Polarity,compound,positive,negative,neutral,Label
0,11432.089844,11759.959961,11388.040039,212830000,0.267549,-0.048568,-0.9982,0.041,0.235,0.724,0
1,11729.669922,11867.110352,11675.530273,183190000,0.374806,0.121956,-0.9858,0.089,0.191,0.721,1
2,11781.700195,11782.349609,11601.519531,173590000,0.536234,-0.044302,-0.9715,0.056,0.128,0.816,0
3,11632.809570,11633.780273,11453.339844,182550000,0.364021,0.011398,-0.9809,0.066,0.146,0.788,0
4,11532.070313,11718.280273,11450.889648,159790000,0.375099,0.040677,-0.9882,0.094,0.189,0.717,1
...,...,...,...,...,...,...,...,...,...,...,...
1984,17355.210938,17355.210938,17063.080078,138740000,0.352311,-0.014015,-0.9644,0.094,0.148,0.758,0
1985,17190.509766,17409.720703,17190.509766,112190000,0.352649,0.046560,-0.9571,0.102,0.132,0.767,1
1986,17456.019531,17704.509766,17456.019531,106380000,0.389617,0.052622,-0.9975,0.091,0.225,0.684,1
1987,17712.759766,17930.609375,17711.800781,133030000,0.382566,0.011243,-0.9977,0.061,0.202,0.738,1


In [ ]:
#Create the feature data set
X=df
X=np.array(X.drop(['Label'],1))
#create target data set
Y=np.array(df['Label'])

In [ ]:
#split the data into 80% training 20% testing
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.2,random_state=0)

In [ ]:
 models = {  'LinearDiscriminantAnalysis':LinearDiscriminantAnalysis(),
                'SVM Classification': SVC(),
                'SGDClassifier': SGDClassifier(loss="hinge", penalty="l2", max_iter=100),
                'KNeighborsClassifier':KNeighborsClassifier(n_neighbors=10),
                'GaussianProcessClassifier': GaussianProcessClassifier(),
                'RandomForestClassifier': RandomForestClassifier(n_estimators=100)
              }

In [ ]:
#show models predictions
for model_name in models.keys():

        model = models[model_name]
        print('--------------',model_name,'---------------')
        print()
        model.fit(x_train,y_train)
        print('--------------','model_prediction','---------------')
        print(model.predict(x_test))
        print()
        print('--------------','classification_report','---------------')
        print(classification_report(model.predict(x_test),y_test))

-------------- LinearDiscriminantAnalysis ---------------

-------------- model_prediction ---------------
[1 1 0 0 1 1 0 0 0 0 0 1 0 1 0 1 1 1 0 1 0 1 0 1 1 0 1 0 1 0 1 0 0 1 0 1 1
 1 1 1 0 0 1 0 1 1 0 1 1 0 1 1 0 0 1 1 1 1 0 1 0 1 1 1 0 1 0 1 0 0 0 0 0 1
 1 1 0 0 1 0 0 1 0 0 1 1 1 1 1 1 0 0 0 0 0 1 0 1 1 0 1 0 1 0 0 0 1 0 0 0 0
 0 1 0 1 1 0 0 1 0 0 0 1 1 0 1 0 1 1 0 1 1 0 0 1 1 0 0 0 1 0 1 0 0 1 1 1 1
 1 0 0 1 0 1 0 0 1 0 1 0 0 1 1 1 1 0 1 1 1 1 1 0 1 0 1 0 1 1 0 1 1 1 1 0 0
 0 1 0 1 1 1 0 1 1 0 0 0 1 1 0 0 1 0 1 1 0 0 0 0 0 1 1 1 1 0 1 0 0 0 1 0 1
 1 0 0 1 1 0 1 0 1 0 0 1 1 1 1 1 0 1 1 0 0 1 1 0 1 1 0 1 1 0 1 1 0 0 0 0 0
 1 1 0 0 0 1 0 0 0 1 1 1 0 0 0 1 1 1 1 0 1 0 1 1 1 0 0 0 0 1 1 1 1 1 1 0 0
 1 0 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 0 1 1 0 1 0 1 1 1 1 1 1 1 1 0 1 1 1 0 1
 0 0 1 1 1 0 0 0 1 1 0 0 1 1 0 1 1 0 1 1 1 1 1 1 1 1 0 0 0 0 0 0 1 1 1 1 1
 0 1 1 0 0 0 1 0 1 1 0 0 1 1 0 0 0 1 0 1 1 1 1 1 1 1 0 0]

-------------- classification_report ---------------
              precision    reca

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


-------------- model_prediction ---------------
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]

-------------- classification_report ---------------
              precision    recall  f1-score   support

           0       1.00      0.48  

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


-------------- model_prediction ---------------
[1 1 0 0 1 1 0 1 1 1 0 1 0 1 0 1 0 0 1 1 0 1 1 1 0 0 0 0 1 0 1 0 1 1 0 0 0
 0 1 1 0 0 1 1 1 0 0 1 1 1 1 1 0 0 1 1 1 0 0 1 0 0 0 1 0 1 1 1 1 0 1 1 1 1
 1 1 0 0 0 0 1 1 1 0 1 1 1 0 1 1 1 1 1 1 0 1 0 1 0 0 1 1 1 1 1 1 1 1 1 1 1
 0 1 1 1 1 0 1 1 1 0 0 1 1 0 1 0 0 1 0 1 1 1 0 1 1 0 1 1 1 1 1 1 1 0 1 1 1
 1 1 0 1 1 1 1 0 1 1 1 1 0 0 1 1 0 1 1 1 0 0 1 0 0 0 1 0 1 0 0 1 1 1 0 0 0
 1 1 0 1 1 1 0 1 1 0 0 0 1 1 0 1 1 1 1 1 1 1 1 1 0 1 0 0 1 1 1 0 0 0 1 1 0
 1 1 1 1 0 1 1 1 0 1 1 1 1 0 1 0 0 0 1 0 1 1 0 1 1 1 0 0 1 1 1 0 1 0 1 0 1
 0 1 1 0 1 0 0 1 0 1 1 0 1 1 1 1 1 0 1 1 0 0 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1
 0 0 0 1 0 0 0 1 0 1 1 0 0 1 1 1 1 1 0 1 1 0 1 1 1 0 1 1 1 1 1 0 0 1 0 1 0
 1 0 0 1 0 0 1 1 1 0 1 0 1 1 0 1 1 0 0 0 1 1 1 0 1 1 1 1 1 0 0 0 0 0 0 1 1
 1 1 1 1 1 0 1 0 0 0 0 0 1 1 1 0 0 0 0 1 1 1 0 1 1 1 0 1]

-------------- classification_report ---------------
              precision    recall  f1-score   support

           0       0.45      0.58  